# 🚀 Training Model Klasifikasi Sampah

Impor Library dan Konfigurasi Utama

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB4, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import os
from pathlib import Path
import json
from datetime import datetime
import shutil

# --- Konfigurasi Utama ---
class Config:
    DATASET_DIR = "./dataset"
    IMG_SIZE = 224
    BATCH_SIZE = 16
    USE_EFFICIENTNET = True 
    DROPOUT_RATE = 0.4
    INITIAL_EPOCHS = 30
    FINE_TUNE_EPOCHS = 50
    INITIAL_LR = 1e-3
    VALIDATION_SPLIT = 0.2
    SAVE_DIR = "./models_output"

config = Config()
os.makedirs(config.SAVE_DIR, exist_ok=True)

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow Version: 2.10.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
def prepare_dataset():
    source_dir = config.DATASET_DIR
    organized_dir = os.path.join(os.path.dirname(source_dir), "dataset_organized")

    if os.path.exists(organized_dir):
        print(f"Menghapus direktori lama: {organized_dir}")
        shutil.rmtree(organized_dir)

    print(f"Membuat struktur direktori baru di: {organized_dir}")
    
    train_dir = os.path.join(organized_dir, 'train')
    val_dir = os.path.join(organized_dir, 'validation')
    
    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    
    for cls in classes:
        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for cls in classes:
        files = [f for f in os.listdir(os.path.join(source_dir, cls)) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        np.random.shuffle(files)
        
        split_index = int(len(files) * (1 - config.VALIDATION_SPLIT))
        train_files = files[:split_index]
        val_files = files[split_index:]

        for f in train_files:
            shutil.copy(os.path.join(source_dir, cls, f), os.path.join(train_dir, cls, f))
        for f in val_files:
            shutil.copy(os.path.join(source_dir, cls, f), os.path.join(val_dir, cls, f))
            
    print("✅ Dataset berhasil diorganisir.")
    return organized_dir

# Jalankan fungsi persiapan dataset
organized_data_dir = prepare_dataset()

Menghapus direktori lama: .\dataset_organized
Membuat struktur direktori baru di: .\dataset_organized
✅ Dataset berhasil diorganisir.


In [19]:
def create_data_generators(data_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),
        target_size=(config.IMG_SIZE, config.IMG_SIZE),
        batch_size=config.BATCH_SIZE,
        class_mode='binary',
        shuffle=True
    )

    validation_generator = validation_datagen.flow_from_directory(
        os.path.join(data_dir, 'validation'),
        target_size=(config.IMG_SIZE, config.IMG_SIZE),
        batch_size=config.BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    
    return train_generator, validation_generator

# Jalankan fungsi untuk membuat generator
train_gen, val_gen = create_data_generators(organized_data_dir)
print(f"Kelas ditemukan: {list(train_gen.class_indices.keys())}")

Found 18051 images belonging to 2 classes.
Found 4513 images belonging to 2 classes.
Kelas ditemukan: ['anorganik', 'organik']


In [20]:
def build_model(num_classes=1):
    if config.USE_EFFICIENTNET:
        base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(config.IMG_SIZE, config.IMG_SIZE, 3))
        print("🚀 Menggunakan arsitektur EfficientNetB4.")
    else:
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(config.IMG_SIZE, config.IMG_SIZE, 3))
        print("⚡ Menggunakan arsitektur MobileNetV2.")

    base_model.trainable = False

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(config.DROPOUT_RATE),
        layers.Dense(num_classes, activation='sigmoid')
    ])
    
    return model

# Bangun modelnya
model = build_model()
model.summary()

🚀 Menggunakan arsitektur EfficientNetB4.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb4 (Functional)  (None, 7, 7, 1792)       17673823  
                                                                 
 global_average_pooling2d_2   (None, 1792)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization_2 (Batc  (None, 1792)             7168      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 256)               459008    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Den

In [23]:
def get_callbacks(model_name):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_path = os.path.join(config.SAVE_DIR, f"{model_name}_{timestamp}.h5")
    
    return [
        keras.callbacks.ModelCheckpoint(filepath=model_path, monitor='val_accuracy', save_best_only=True, verbose=1),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1)
    ], model_path

print("\n--- TAHAP 1: Melatih Head Model (Feature Extraction) ---")
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=config.INITIAL_LR),
    loss='binary_crossentropy',
    metrics=['accuracy'] # PERBAIKAN: Hanya gunakan 'accuracy' di sini untuk menghindari error serialisasi
)

callbacks, _ = get_callbacks("waste_model_initial_head")

history = model.fit(
    train_gen,
    epochs=config.INITIAL_EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks
)


--- TAHAP 1: Melatih Head Model (Feature Extraction) ---
Epoch 1/30


ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_2/efficientnetb4/block4e_expand_bn/FusedBatchNormV3' defined at (most recent call last):
    File "C:\Users\asus\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\asus\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "d:\waste-classifier\server\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "d:\waste-classifier\server\venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "d:\waste-classifier\server\venv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "d:\waste-classifier\server\venv\lib\site-packages\tornado\platform\asyncio.py", line 211, in start
      self.asyncio_loop.r

In [ ]:
print("\n--- TAHAP 2: Melatih Sebagian Layer (Fine-Tuning) ---")
base_model = model.layers[0]
base_model.trainable = True

fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=config.INITIAL_LR / 10),
    loss='binary_crossentropy',
    metrics=['accuracy'] # PERBAIKAN: Hanya gunakan 'accuracy' di sini
)

callbacks_finetune, best_model_path = get_callbacks("high_accuracy_waste_classifier")

history_fine = model.fit(
    train_gen,
    epochs=config.INITIAL_EPOCHS + config.FINE_TUNE_EPOCHS,
    initial_epoch=history.epoch[-1],
    validation_data=val_gen,
    callbacks=callbacks_finetune
)

print("\n🎉 Training Selesai!")
print(f"💾 Model terbaik disimpan di: {best_model_path}")

In [ ]:
print("\n--- Memulai Evaluasi Akhir pada Model Terbaik ---")
best_model = keras.models.load_model(best_model_path)

val_gen.reset()
predictions = best_model.predict(val_gen, verbose=1)
predicted_classes = (predictions > 0.5).astype(int).flatten()
true_classes = val_gen.classes
class_names = list(val_gen.class_indices.keys())

print("\n📊 Laporan Klasifikasi:\n" + "-"*40)
print(classification_report(true_classes, predicted_classes, target_names=class_names, digits=4))

cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('Label Sebenarnya')
plt.xlabel('Label Prediksi')
plt.savefig(os.path.join(config.SAVE_DIR, 'confusion_matrix.png'))
plt.show()

# Plotting training history
all_history = {
    'accuracy': history.history['accuracy'] + history_fine.history['accuracy'],
    'val_accuracy': history.history['val_accuracy'] + history_fine.history['val_accuracy'],
    'loss': history.history['loss'] + history_fine.history['loss'],
    'val_loss': history.history['val_loss'] + history_fine.history['val_loss'],
}

plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.plot(all_history['accuracy'], label='Training Accuracy')
plt.plot(all_history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(all_history['loss'], label='Training Loss')
plt.plot(all_history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(config.SAVE_DIR, 'training_history.png'))
plt.show()
